<a href="https://colab.research.google.com/github/LeanKhan/quiztory/blob/main/NG_History_Timeline_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Algorithm:

- Break down text into Parts-Of-Speech tags
- Extract the VBD (past tense verbs _happened_) in the sentence if any
- Get base for that word _happen_. This is the thing that "happened" on that Date
- Display the Named Entities using NER and extract the Date entity. 
- **Construct question**
- Remove date and prepositions like "in, on" before date.
- Add "When " to beginning of question. Replace VBD with base form.
- *profit* ─=≡Σ((( つ◕ل͜◕)つ

Install deps

In [1]:
import spacy

# import nltk
import re
import requests

# from nltk.tokenize import word_tokenize
# from nltk.tag import pos_tag
# from nltk.stem import WordNetLemmatizer

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('words')
# nltk.download('wordnet')
# nltk.download('maxent_ne_chunker')

# install wikipedia article api wrapper
import sys
!{sys.executable} -m pip install wikipedia
!{sys.executable} -m pip install dateparser
!{sys.executable} -m pip install daterangeparser

# fast api
!{sys.executable} -m pip install colabcode
!{sys.executable} -m pip install fastapi
!{sys.executable} -m pip install jinja2

# node
!npm install -g localtunnel

# import installed deps
import wikipedia as wp
from dateparser import parse
from dateparser import parse as parse_range



/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.276s


   ╭───────────────────────────────────────────────────────────────╮
   │                                                               │
   │      New major version of npm available! 6.14.8 → 8.5.1       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v8.5.1   │
   │               Run npm install -g npm to update!               │
   │                                                               │
   ╰───────────────────────────────────────────────────────────────╯



Setup Server

Initialize text

In [2]:
def extract_all_events(text, extract_function):
  all_events = []
  questions = []
  processed = 0
  # Process the events
  for processed,line in enumerate(text.splitlines()):
    events = extract_function(line)
    all_events = all_events + events
    if processed % 100 == 0:
      print('Processed: {}'.format(processed))

  print("Extracted {} events.".format(len(all_events)))
  # use event[2] to forumulate the question...

  # Print out the events
  for event in sorted(all_events, key=lambda e: e[0]):
    questions.append({"q": "When did {}?".format(event[2]), "ans": event[1]})
  
  return questions

Get NER and get date as well as preposition

Construct question

Dependency Parsing, thank you Jesus!

In [3]:
def extract_events_regex(line):
  matches = []
  # capture thee digit and four digit years (1975) and ranges (1975-1976)
  found = re.findall('In (\d\d\d\d?[/\–]?\d?\d?\d?\d?),? ?([^\\.]*)', line)
  try:
    matches = matches + list(map(lambda f: (f[0] if len(f[0])>3 else "0"+f[0] ,f[0],f[1]),found))
  except:
   return []
  return matches

def dep_subtree(token, dep):
  deps =[child.dep_ for child in token.children]
  child=next(filter(lambda c: c.dep_==dep, token.children), None)
  if child != None:
    return " ".join([c.text for c in child.subtree])
  else:
    return ""

# to remove citations, e.g. "[91]" as this makes problems with spaCy
p = re.compile(r'\[\d+\]')

def extract_events_spacy(line):
  line=p.sub('', line)
  events = []
  doc = nlp(line)
  for ent in filter(lambda e: e.label_=='DATE',doc.ents):
    # print(ent.text)
    try:
      start, end = parse_range(ent.text)
      # start = ent.text
    except Exception as e:
      # continue
      # could not parse the dates, hence ignore it
      try:
        # start = parse(ent.text)
        start = ent.text
      except:
        print('Could not => ', e)
        continue

    current = ent.root
    while current.dep_ != "ROOT":
      current = current.head
    desc = " ".join(filter(None,[
                                 dep_subtree(current,"nsubj"),
                                 dep_subtree(current,"nsubjpass"),
                                 dep_subtree(current,"auxpass"),
                                 dep_subtree(current,"amod"),
                                 dep_subtree(current,"det"),
                                 current.text, 
                                 dep_subtree(current,"acl"),
                                 dep_subtree(current,"dobj"),
                                 dep_subtree(current,"attr"),
                                 dep_subtree(current,"advmod")]))
    events = events + [(start,ent.text,desc)]
  return events

All together now!

In [ ]:
# extract_all_events(summary,extract_events_spacy)

# extract_events_spacy("The Protestant Reformation was the first successful challenge to the Catholic Church and began in 1521 - 1600 as Luther was outlawed at the Diet of Worms after his refusal to repent. ")


Get templates! Thank you Jesus!

In [ ]:
# raw_text="""Nigeria gained her independence in 1960."""
# raw_text = """Frank Roffolo was killed on December 2019"""
# response = requests.get('https://raw.githubusercontent.com/qualicen/timeline/master/history_of_germany.txt')

# summary = wp.summary("Colonial Nigeria")
# summary = response.content

# # Create NER text
# text = NER(raw_text)

# lem = WordNetLemmatizer()


In [4]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
from jinja2 import Template

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.on_event("startup")
def load_model():
    global nlp
    nlp = spacy.load("en_core_web_sm")

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}

@app.get('/home', response_class=HTMLResponse)
def home():
    template_string = requests.get('https://raw.githubusercontent.com/LeanKhan/congenial-lamp/main/index.html').text

    template = Template(template_string)

    return template.render()


@app.get('/questions', response_class=HTMLResponse)
def questions(article: str):
    template_string = requests.get('https://raw.githubusercontent.com/LeanKhan/congenial-lamp/main/questions.html').text

    template = Template(template_string)

    global wp_page

    try:
      wp_page = wp.page(article)
    except:
      # raise error
      return {"error": True}
    
    # doc = nlp(summary)

    qs = extract_all_events(wp_page.summary,extract_events_spacy)

    return template.render(page=wp_page, questions=qs)


@app.get('/api/questions')
def get_questions(article: str):

    global wp_page

    try:
      wp_page = wp.page(article)
    except:
      # raise error
      return {"error": True}
    
    # doc = nlp(summary)

    qs = extract_all_events(wp_page.summary,extract_events_spacy)

    return {'success': True, 'questions': qs, 'links': wp_page.links, 'images': wp_page.images, 'title': wp_page.title, 'url': wp_page.url}




Run Server! Thank you Jesus!

In [ ]:
from colabcode import ColabCode

server = ColabCode(port=10000, code=False)

server.run_app(app=app)

# This setup is to create a publicly available url to view the site. Unfortunately, you need an auth token to return HTML from
# an NGROK tunnel nowadays... But you can still interact with the api using Postman or similar.

# after server starts try entering this address in your address bar: {{the Public url}}/api/questions?article=Colonial%20Nigeria


INFO:     Started server process [266]
INFO:     Waiting for application startup.


Public URL: NgrokTunnel: "https://86d3-34-125-1-108.ngrok.io" -> "http://localhost:10000"


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     197.210.71.107:0 - "GET / HTTP/1.1" 200 OK
INFO:     197.210.71.107:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     197.210.71.107:0 - "GET /home HTTP/1.1" 200 OK


Exception in callback BaseAsyncIOLoop._handle_events(18, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(18, 1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/nest_asyncio.py", line 203, in run
    self._context.run(update_from_context, ctx)
RuntimeError: cannot enter context: <Context object at 0x7fd529f54fa0> is already entered
Exception in callback BaseAsyncIOLoop._handle_events(18, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(18, 1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/nest_asyncio.py", line 203, in run
    self._context.run(update_from_context, ctx)
RuntimeError: cannot enter context: <Context object at 0x7fd529f54fa0> is already entered
